In [ ]:
import math

c_dict = {
    4: 64,
    6: 24
}

cidr_max = {
    4: 28,
    6: 48
}

def __get_min_samples(ip_version, mask):

    c= c_dict[ip_version]

    
    if ip_version == 4:
        ipv_max = 32
        min_samples=int(c * math.sqrt( math.pow(2, (ipv_max - mask))))
    elif ip_version == 6:
        ipv_max = 64
        min_samples=int(c * math.sqrt(math.sqrt( math.pow(2, (ipv_max - mask)))))
    else:
        pass

    # TODO für später
    

    # bei v6 doppelte wurzel

    # self.logger.info(f"min samples: {min_samples}")
    return min_samples

for i in range(0, cidr_max[4]):
    print(f"4 {i}:\t{__get_min_samples(6,i)}")

for i in range(0, cidr_max[6]):
    print(f"6 {i}:\t{__get_min_samples(6,i)}")

In [12]:
from netaddr import IPNetwork, IPAddress
import pytricia
from collections import defaultdict

def join_siblings(ip_version, mask, prange):

    ## check if join would be possible
    if mask == 0:
        print("        join siblings not possible - we are at the root of the tree")
        return None


    nw = IPNetwork(f"{prange}/{mask}")


    #### GET SUPERNET
    nw_supernet=nw.supernet(mask-1)[0]
    supernet_ip=str(nw_supernet).split("/")[0]
    supernet_mask=int(str(nw_supernet).split("/")[1])

    siblings=list(nw_supernet.subnet(mask))
    print(f"join_siblings supernet: {nw_supernet}")
    print(f" siblings: {siblings}")

join_siblings(6,33,"2a05:d018:1::" )

p = pytricia.PyTricia(128)
p.insert("0:0:2::/48","0:0:2::/48",)
p.insert("2a05:d018::/33", "2a05:d018::/33")
p.insert("2a05:d018:8000::/33", "2a05:d018:8000::/33")


print("lpm: ", list(p))

for i in ["2a05:d018:1::", "2a05:d018:dc0::", "2a05:d018:76c::", "2a05:d018:5::", "2a05:d018:4e8::"]:
    try:
        print(p[i])
    except:
        print(f"prefix not found: {i}")


d = defaultdict(int)
d["0:0:2::/48"] = 5

print(d)
d.pop("0:0:2::/48",)


join_siblings supernet: 2a05:d018::/32
 siblings: [IPNetwork('2a05:d018::/33'), IPNetwork('2a05:d018:8000::/33')]
lpm:  ['0:0:2::/48', '2a05:d018:8000::/33', '2a05:d018::/33']
prefix not found: 2a05:d018:1::
prefix not found: 2a05:d018:dc0::
prefix not found: 2a05:d018:76c::
prefix not found: 2a05:d018:5::
prefix not found: 2a05:d018:4e8::
defaultdict(<class 'int'>, {'0:0:2::/48': 5})


5

In [ ]:
import datetime
import matplotlib.pyplot as plt

#logfile="/home/mehneste/ipd_algo/log/q0.95_c64-24_cidr_max28-48_t60_e120_decaydefault.log" # read rb
logfile="/home/mehneste/ipd_algo/log/q0.95_c64-24_cidr_max28-48_t60_e120_decaydefault.log" # read rt 


### data stuff

format = "%Y-%m-%d %H:%M:%S,%f"

file = open(logfile, 'r')
Lines = file.readlines()

count=0
last_start_ts=None
start_ts=None
end_ts=None

ipd_run_list=[]
iteration_run_list=[]
prefix_list=[]
for line in Lines:
    count += 1
    # print("Line{}: {}".format(count, line.strip()))
    x = line.split(" ")

    ts=datetime.datetime.strptime(f"{x[0]} {x[1]}", format)
    #print(ts)
    if last_start_ts == None:   last_start_ts = ts
    if "run IPD" in line:
        start_ts=ts

        try:
            prefix_list.append(int(x[-2]))
        except:
            pass

        if last_start_ts != start_ts:
            iteration_duration= start_ts- last_start_ts
            iteration_run_list.append(iteration_duration.seconds)
            last_start_ts = start_ts
        
    elif "Finished" in line:
        end_ts=ts
        duration = end_ts - start_ts
        ipd_run_list.append(duration.seconds)

## plotting stuff

fig,ax = plt.subplots()
ax.plot(ipd_run_list, label="IPD run duration only")
ax.plot(iteration_run_list, label="Netflow read + IPD")


ax.set_xlabel("Netflow minute")
ax.set_ylabel("duration in seconds")
plt.legend()

ax2=ax.twinx()
ax2.plot(prefix_list, label="prefix count", linestyle="dotted", color="grey")
ax2.set_ylabel("prefix count")



plt.suptitle(" with gzip.open()")
print(f"currently {len(iteration_run_list)} minutes done")
print(f"IPD run: avg: {sum(ipd_run_list) / len(ipd_run_list)}s; min: {min(ipd_run_list)}; max: {max(ipd_run_list)}")
print(f"iteration run: avg: {sum(iteration_run_list) / len(iteration_run_list)}s; min: {min(iteration_run_list)}s; max: {max(iteration_run_list)}s")




In [ ]:
import datetime
from sys import prefix
import matplotlib.pyplot as plt

logfile="/home/mehneste/ipd_algo/log_stdin/q0.95_c64-24_cidr_max28-48_t60_e120_decaydefault.log" # read rb
#logfile="/home/mehneste/ipd_algo/log_rt/q0.95_c64-24_cidr_max28-48_t60_e120_decaydefault.log" # read rt 


### data stuff

format = "%Y-%m-%d %H:%M:%S,%f"

file = open(logfile, 'r')
Lines = file.readlines()

count=0
last_start_ts=None
start_ts=None
end_ts=None

ipd_run_list=[]
iteration_run_list=[]
prefix_list=[]
for line in Lines:
    count += 1
    # print("Line{}: {}".format(count, line.strip()))
    x = line.split(" ")

    ts=datetime.datetime.strptime(f"{x[0]} {x[1]}", format)
    #print(ts)
    if last_start_ts == None:   last_start_ts = ts
    if "run IPD" in line:
        start_ts=ts
        try:
            prefix_list.append(int(x[-2]))
        except:
            pass
        if last_start_ts != start_ts:
            iteration_duration= start_ts- last_start_ts
            iteration_run_list.append(iteration_duration.seconds)
            last_start_ts = start_ts
        
    elif "Finished" in line:
        end_ts=ts
        duration = end_ts - start_ts
        ipd_run_list.append(duration.seconds)

## plotting stuff

fig,ax = plt.subplots()
ax.plot(ipd_run_list, label="IPD run duration only")
ax.plot(iteration_run_list, label="Netflow read + IPD")


ax.set_xlabel("Netflow minute")
ax.set_ylabel("duration in seconds")
plt.legend()

ax2=ax.twinx()
ax2.plot(prefix_list, label="prefix count", linestyle="dotted", color="grey")
ax2.set_ylabel("prefix count")


plt.suptitle("zcat | sys.stdin")

print(f"currently {len(iteration_run_list)} minutes done")
print(f"IPD run: avg: {sum(ipd_run_list) / len(ipd_run_list)}s; min: {min(ipd_run_list)}; max: {max(ipd_run_list)}")
print(f"iteration run: avg: {sum(iteration_run_list) / len(iteration_run_list)}s; min: {min(iteration_run_list)}s; max: {max(iteration_run_list)}s")


In [5]:
import datetime
from sys import prefix
import matplotlib.pyplot as plt
import  glob
import numpy as np



### data stuff

format = "%Y-%m-%d %H:%M:%S,%f"

for logfile in glob.glob("/home/mehneste/ipd_algo/log/*"):
    file = open(logfile, 'r')
    Lines = file.readlines()

    count=0
    last_start_ts=None
    start_ts=None
    end_ts=None

    ipd_run_list=[]
    iteration_run_list=[]
    prefix_list=[]
    for line in Lines:
        count += 1
        # print("Line{}: {}".format(count, line.strip()))
        x = line.split(" ")

        ts=datetime.datetime.strptime(f"{x[0]} {x[1]}", format)
        #print(ts)
        if last_start_ts == None:   last_start_ts = ts
        if "run IPD" in line:
            start_ts=ts
            try:
                prefix_list.append(int(x[-2]))
            except:
                pass

            if last_start_ts != start_ts:
                iteration_duration= start_ts- last_start_ts
                if int(iteration_duration.seconds) > 600:
                    #print(x)
                    print(f"Caution: {last_start_ts} -> {start_ts}: netflow ts: {int(x[-5])} iteration run took extremly long: {iteration_duration}")
                iteration_run_list.append(iteration_duration.seconds)
                last_start_ts = start_ts
            
        elif "Finished" in line:
            end_ts=ts
            duration = end_ts - start_ts
            ipd_run_list.append(duration.seconds)

    ## plotting stuff
    # get t
    t = int(logfile.split("_t")[1].split("_e")[0])/60
    

    fig,ax = plt.subplots()
    ax.plot(ipd_run_list, label="IPD run duration only")
    ax.plot(iteration_run_list, label="Netflow read + IPD")
    # ax.set_xticklabels(np.arange(0, 1560, t))

    ax.set_xlabel(f"Netflow bucket (t= {int(t*60)}s)")
    ax.set_ylabel("duration in seconds")
    ax.set_ylim(0, 500)
    ax.set_xlim(0, 1560/t)
    
    plt.legend()

    ax2=ax.twinx()
    ax2.plot(prefix_list, label="prefix count", linestyle="dotted", color="grey")
    ax2.set_ylabel("prefix count")
    ax2.set_ylim(0, 800000)


    plt.suptitle(f"{logfile}")

    print(f"currently {len(iteration_run_list)*t} minutes done")
    print(f"IPD run: avg: {sum(ipd_run_list) / len(ipd_run_list)}s; min: {min(ipd_run_list)}; max: {max(ipd_run_list)}")
    print(f"iteration run: avg: {sum(iteration_run_list) / len(iteration_run_list)}s; min: {min(iteration_run_list)}s; max: {max(iteration_run_list)}s")
    plt.show()


KeyboardInterrupt: 